In [1]:
import os
import glob
import json
import numpy as np
import pandas as pd
import geopandas as gpd
import clima_anom as ca

import matplotlib.cbook
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import cartopy
import cartopy.feature as cf
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader

/home/arturo/anaconda3/envs/AXE/lib/python3.9/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.11.0-CAPI-1.17.0) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
# json_method = 'AMA_pearson'
# json_method = 'AMA_spearman'
# json_method = 'AMA_kendall'
json_method = 'SA_sync'

In [3]:
with open ('../../json/directories.json') as f:
    dirs = json.load(f)

json_read = dirs['json_CN'] + json_method+'.json'
print(f'Read json: {json_read}')

with open (json_read) as f:
    info = json.load(f)

Read json: ../../../Complex-Network-AS/json/SA_sync.json


In [4]:
product = info['product']
season = info['season']
region = info['region']
pentad = info['pentad']
ocean = info['ocean']
method = info['method']
centrado = info['centrado']
normalized = info['normalized']

percentile_limit = info['percentile_limit']
tau = info['tau']

min_mean = info['remove_serie_min_mean']
MLV = info['Percent_links_ignore']
rain_LT1 = info['remove_rain_LT_1']

# font = os.environ['COMPUTERNAME']

# print(f'Font      : {font}')
print(f'Product   : {product}')
print(f'Region   : {region}')
print()
print(f'Season    : {season}')
print(f'Pentad    : {pentad}')
print(f'Ocean     : {ocean}')
print()
print(f'Method    : {method}')
print()
print(f'Tau       : {tau}')
print()
print(f'Percentile Limit: {percentile_limit}')
print(f'Centered        : {centrado}')
print(f'Normalized      : {normalized}')
print()
print(f'Remove Serie with min mean : {min_mean}')
print(f'Remove Serie with rain < 1 : {rain_LT1}')
print(f'Minimun link value (MLV)   : {MLV}%')

if rain_LT1 == 'yes':
    LT_1 = 'LT1Y'
else:
    LT_1 = 'LT1N'

if percentile_limit == 90 or percentile_limit == 95 or percentile_limit == 99:
    PL_limit = percentile_limit
else:
    PL_limit = 'N'

if ocean == 'woc':
    OC = 'WOC'
else:
    OC = 'NOC'

if tau == 'no':
    TA = 'TAUN'
else:
    TA = 'TAU'+str(tau)

Product   : IMERG
Region   : SA

Season    : summer
Pentad    : no
Ocean     : noc

Method    : sync

Tau       : 3

Percentile Limit: 90
Centered        : no
Normalized      : no

Remove Serie with min mean : 1
Remove Serie with rain < 1 : yes
Minimun link value (MLV)   : 98%


In [5]:
if os.environ['COMPUTERNAME'] == 'DELL_LAPTOP':
    export_dir = 'D:/Data/PF/ENSEMBLE/ENSEMBLE_SA/' + method + '/'
    PF_ensemble_files = 'D:\Data\PF\ENSEMBLE\list\TRMM_GPM_summer_200012_2021_02.csv'
else:
    export_dir = dirs['dir_ENSEMBLE'] + method + '/'
    PF_ensemble_files = dirs['dir_ENSEMBLE_list']

print(os.environ['COMPUTERNAME'])
print(f'Export dir: {export_dir}')

idell
Export dir: /mnt/Data/Data/PF/ENSEMBLE/ENSEMBLE_SA/sync/


In [6]:
comms_file = region+'_'+product+'_05x05_'+method+'_'+season+'_'+LT_1+'_PL'+str(PL_limit)+'_PEN_MM'+str(min_mean)+'_MLV'+str(100-MLV)+'_'+TA
print(f'Comms file: {comms_file}')

Comms file: SA_IMERG_05x05_sync_summer_LT1Y_PL90_PEN_MM1_MLV2_TAU3


In [7]:
file_list = pd.read_csv(PF_ensemble_files + 'TRMM_GPM_summer_200012_2021_02.csv')
len_files = len(file_list)
print(f'Files Number: {len_files}')
print(file_list['file_name'].iloc[0])
print(file_list['file_name'].iloc[-1])

Files Number: 63
/mnt/Data/Data/PF/TRMM/rpf_SA/rpf_200012_level2.csv
/mnt/Data/Data/PF/GPM/rgpf_SA/rgpf_202102_level2.csv


In [8]:
communities_poligon = gpd.read_file(dirs['dir_comm'] + comms_file + '.geojson')
communities_index = communities_poligon['id'].values.astype(int)
communities_index

array([1, 2])

In [9]:
Comunidad_file = dirs['dir_comm'] + comms_file + '.shp'
Comunidades = gpd.read_file(Comunidad_file)
num_comm = len(Comunidades)
print(f'Number of Communities: {num_comm}')

Number of Communities: 2


In [10]:
# test names
comm_id = communities_index[0]
comm_name = comms_file + '_C'+str(comm_id).zfill(3)
salida = export_dir + comm_name + '.csv'
salida

'/mnt/Data/Data/PF/ENSEMBLE/ENSEMBLE_SA/sync/SA_IMERG_05x05_sync_summer_LT1Y_PL90_PEN_MM1_MLV2_TAU3_C001.csv'

In [11]:
for comm_id in range(num_comm):

    comm_id_new = communities_index[comm_id]
    print(f'Community: {comm_id}')

    for nn in range(len_files):

        if os.environ['COMPUTERNAME'] == 'DELL_LAPTOP':
            data = pd.read_csv(file_list['file_name'][nn].replace('/mnt/Data/Data/PF/','D:/Data/PF/'))
        else:
            data = pd.read_csv(file_list['file_name'][nn], sep=',')

        data[data==-88.88] = np.nan
        data[data==-99.99] = np.nan

        PF_GPD = gpd.GeoDataFrame(data[data.columns[2:]], geometry=gpd.points_from_xy(data['lon'],data['lat'], crs="EPSG:4326"))
        num_points = len(PF_GPD['geometry']) 

        points_interception = [] 

        for t in range(num_points):

            if Comunidades['geometry'][comm_id].contains(PF_GPD['geometry'][t]) == True:
                points_interception.append(t)

        dic_tmp = data.loc[points_interception]
        dic_tmp = dic_tmp.reset_index(drop=True)

        if nn == 0:
            dict_out = dic_tmp

        else:
            dict_out = pd.concat([dic_tmp, dict_out], axis=0)

        # print(f'Points in the community: {len(dic_tmp)}')

    comm_name = comms_file + '_C'+str(comm_id_new).zfill(3)
    salida = export_dir + comm_name + '.csv'
    print(f'Exporting data to {salida}')
    dict_out.to_csv(salida,header=True,index=False)

Community: 0
Exporting data to /mnt/Data/Data/PF/ENSEMBLE/ENSEMBLE_SA/sync/SA_IMERG_05x05_sync_summer_LT1Y_PL90_PEN_MM1_MLV2_TAU3_C001.csv
Community: 1
Exporting data to /mnt/Data/Data/PF/ENSEMBLE/ENSEMBLE_SA/sync/SA_IMERG_05x05_sync_summer_LT1Y_PL90_PEN_MM1_MLV2_TAU3_C002.csv


In [12]:
# Comunidad_file = 'Comm_IMERG_05x05_sync_summer_P90_tau3_SP95'
# comm_id = 1
# comm_name = 'C'+str(comm_id+1).zfill(2)
# print(comm_name)

# levels = np.arange(0.5,20.5,0.5)

# for nn in range(63):

#     data = pd.read_csv(file_list['file_name'][nn], sep=',')
#     data[data==-88.88] = np.nan
#     data[data==-99.99] = np.nan

#     PF_GPD = gpd.GeoDataFrame(data[data.columns[2:]], geometry=gpd.points_from_xy(data['lon'],data['lat'], crs="EPSG:4326"))

#     comm_GPD = gpd.read_file(DIRS['dir_communities']'+Comunidad_file+'.shp')

#     num_comm = len(comm_GPD['geometry'])
#     num_points = len(PF_GPD['geometry']) 

#     points_interception = []  

#     for t in range(num_points):

#         if comm_GPD['geometry'][comm_id].contains(PF_GPD['geometry'][t]) == True:
#             points_interception.append(t)

#     dic_tmp = data.loc[points_interception]
#     dic_tmp = dic_tmp.reset_index(drop=True)

#     if nn == 0:
#         dict_out = dic_tmp

#     else:
#         dict_out = pd.concat([dic_tmp, dict_out], axis=0)

#     print(f'Points in the community: {len(dic_tmp)}')

# # export dataframe to csv
# salida = '/mnt/Data/Data/PF/ENSEMBLE/ENSEMBLE_SA/'+Comunidad_file+'_'+comm_name+'.csv'
# print(f'Exporting data to {salida}')
# dict_out.to_csv(salida,header=True,index=False)